In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import time
import random

#### Use the get method to get the html from the url 

In [4]:
url = 'https://www.openthebooks.com/maryland-state-checkbook/?Year_C=2021&C_PG=1'
res = requests.get(url)

Check the status to see if it worked

In [5]:
res.status_code

200

##### Pull the HTML string out of requests and convert it to a Python string.

In [6]:
html = res.text

#### Using BeautifulSoup, parce the document into lxml format

lxml is a Python library which allows for easy handling of XML and HTML files 

<i>https://stackabuse.com/introduction-to-the-python-lxml-library/</i>                                       

In [7]:
soup = BeautifulSoup(html, 'lxml')

In [8]:
table = soup.find('table', {'class':'employer-detail-table'})

In [9]:
type(table)

bs4.element.Tag

#### Lets get the table header

In [10]:
header = []
for head in table.find_all('th'):
    header.append(head.text)

In [11]:
header

['Year', 'Employer', 'Vendor Name', 'Amount', 'Source']

#### Loop through the table object and get each row

In [12]:
#empty list to append dictionary
all_payout = []
for row in table.find_all('tr')[1:]:
    i = 0
    payout = {}
    for cell in row.find_all('td'):
        payout[header[i]] = cell.text.strip()
        i += 1
    all_payout.append(payout)

In [13]:
pd.DataFrame(all_payout)

,Year,Employer,Vendor Name,Amount,Source
0,2021,"Maryland Department Of Health, Mdh",Priority Partners Mco,"$1,713,212,267.00","Maryland Department Of Health, Mdh"
1,2021,Maryland State Department Of Education,Prince Georges County Public School,"$1,401,301,255.00",Maryland State Department Of Education
2,2021,"Maryland Department Of Health, Mdh",Amerigroup Maryland Inc,"$1,343,588,177.00","Maryland Department Of Health, Mdh"
3,2021,"Maryland Department Of Health, Mdh",Maryland Care Inc,"$1,323,068,250.00","Maryland Department Of Health, Mdh"
4,2021,Redemption And Interest On State Bonds,Depository Trust Company,"$1,267,786,597.00",Redemption And Interest On State Bonds
...,...,...,...,...,...
95,2021,Maryland Higher Education Commission,Johns Hopkins University Physicians,"$35,072,306.00",Maryland Higher Education Commission
96,2021,Maryland Department Of Labor,Anne Arundel County Md,"$34,926,838.22",Maryland Department Of Labor
97,2021,Military Department,Montgomery County Maryland,"$34,762,810.35",Military Department
98,2021,Maryland Department Of Labor,Accenture Llp,"$34,025,167.83",Maryland Department Of Labor


Now let's do this for all 10,000 pages

In [14]:
all_pages = []
for i in range(1,3):
    url = 'https://www.openthebooks.com/maryland-state-checkbook/?Year_C=2021&C_PG=' + str(i)
    
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'lxml')
    table = soup.find('table', {'class':'employer-detail-table'})
    
    for row in table.find_all('tr')[1:]:
        count = 0
        payout = {}
        for cell in row.find_all('td'):        
            payout[header[count]] = cell.text.strip()
            count += 1
        all_pages.append(payout)
    time.sleep(5)

In [15]:
df = pd.DataFrame(all_pages)

#### Export

In [20]:
df.to_csv('data/Maryland_State_Checkbook_raw.csv', index=False)